Script for scraping old data from dhhs

In [39]:
import pandas as pd
import calendar
from datetime import date
from datetime import timedelta

date formats on the dhhs website:

05-june-2020
9-june-2020
14-june-2020
21-june
sunday-12-july
thursday-16-july-2020

In [93]:
def get_url(date):
        
    prefix = 'https://www.dhhs.vic.gov.au/coronavirus-update-'
        
    y = date.year
    m = calendar.month_name[date.month].lower()
    d = date.day
    wd = calendar.day_name[date.weekday()].lower()
    
    suffix1 = '0{}-{}-{}'.format(d, m, y)
    suffix2 = '{}-{}-{}'.format(d, m, y)
    suffix3 = '0{}-{}'.format(d, m)
    suffix4 = '{}-{}'.format(d, m)
    suffix5 = '{}-0{}-{}'.format(wd, d, m)
    suffix6 = '{}-{}-{}'.format(wd, d, m)
    suffix7 = '{}-0{}-{}-{}'.format(wd, d, m, y)
    suffix8 = '{}-{}-{}-{}'.format(wd, d, m, y)
    
    print(suffix1, suffix2, suffix3, suffix4, suffix5, suffix6, suffix7, suffix8)

In [94]:
some_date = date(2020, 6, 14)
get_url(some_date)

014-june-2020 14-june-2020 014-june 14-june sunday-014-june sunday-14-june sunday-014-june-2020 sunday-14-june-2020


In [ ]:
curr_date = date(2020, 6, 14)
end_date = date.today()

while (curr_date != end_date):
    curr_date += timedelta(days=1)
    print(curr_date)

In [11]:
url = "https://www.dhhs.vic.gov.au/coronavirus-update-victoria-14-june-2020"
df_list = pd.read_html(url)

In [13]:
df_list[0]

,LGA,Total confirmed cases (ever),Currently active cases
0,MELBOURNE,115,8
1,STONNINGTON,95,1
2,BANYULE,94,2
3,BOROONDARA,73,1
4,BRIMBANK,73,1
...,...,...,...
70,MOYNE,1,0
71,WEST WIMMERA,1,0
72,WODONGA,1,0
73,YARRIAMBIACK,1,0


In [19]:
today = date.today()
print(today.year)
print(today.month)
print(today.day)
print(today.weekday())

2020
7
27
0


In [33]:
tomorrow = today + timedelta(days=1)
print(tomorrow.year)
print(tomorrow.month)
print(tomorrow.day)
print(tomorrow.weekday())
print(weekdays[tomorrow.weekday()])

2020
7
28
1
tuesday


2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-19
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
